In [ ]:
# Point Colaboratory to your Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
datapath = '/content/gdrive/My Drive/^DATA SCIENCE/^Biological Data/PROJECT/models/' # Luca
# datapath = '/content/gdrive/My Drive/Data Science - Unipd/II Year/I Semester/Biological Data/Project/PROJECT/models/' # Giuliano

# Building models

## Ex 5: build a PSSM model

In [ ]:
# ./binx/ncbi-blast-2.11.0+/bin/psiblast -in_msa ./PROJECT/clustal_omega_uniref90_100hits.fasta -db ./data/msa/uniprot_sprot.fasta -out ./PROJECT/output_psiblast_uniref90_100hits.out -out_pssm ./PROJECT/pssm_model_uniref90_100hits.pssm

## Ex 6: build a HMM model

In [ ]:
# ./binx/hmmer-3.3.1/src/hmmbuild ./PROJECT/hmm_model_uniref90_100hits.hmm ./PROJECT/clustal_omega_uniref90_100hits.fasta

## Ex 7: find significant hits using HMM-SEARCH and PSI-BLAST

In [ ]:
# hmmsearch --tblout ./PROJECT/hmm_search_uniref90_100hits.hmmer_tblout --domtblout ./PROJECT/hmm_search_uniref90_100hits.hmmer_domtblout ./PROJECT/hmm_model_uniref90_100hits.hmm ./data/msa/uniprot_sprot.fasta > ./PROJECT/hmm_search_uniref90_100hits.hmmer_align

In [ ]:
# ./binx/ncbi-blast-2.11.0+/bin/psiblast -in_pssm ./PROJECT/pssm_model_uniref90_100hits.pssm -db ./data/msa/uniprot_sprot.fasta -outfmt 6 -num_iterations 4 -evalue 0.01 > ./PROJECT/psiblast_search_uniref90_100hits.txt

## Ex 8

#### Read the ground truth file


In [ ]:
!pip install Bio
from Bio import SeqIO

     |████████████████████████████████| 71kB 3.2MB/s 
     |████████████████████████████████| 2.3MB 5.6MB/s 


Find the proteins containing the target domain (PF08543) in:
- SwissProt (annotated) 
- TrEMBLE (not annotated) ?????

Find the proteins containing the target domain (PF08543) in SwissProt -> set of actual positives 


In [ ]:
# !pip install Bio
# from Bio import SeqIO
path = datapath + 'protein-matching-PF08543.fasta'
protein_matching = SeqIO.parse(path,'fasta')

list_protein_containing_domain = []
for sequence in protein_matching:
    # print(sequence)
    name = sequence.id # name is in the form A1ADT5|reviewed|Pyridoxine/pyridoxal/pyridoxamine
    # print('name ->', name, '\n')
    list_protein_containing_domain.append(name.split('|')[0])

set_protein_containing_domain = set(list_protein_containing_domain) # this is the ground truth
print(f"Number of proteins in SwissProt containing the target domain: {len(list_protein_containing_domain)}")
# this is the total number of protein in SwissProt
print(f"Actual positives: {set_protein_containing_domain}")

Number of proteins in SwissProt containing the target domain: 150
Actual positives: {'P55882', 'B7M6S8', 'Q9ZBR6', 'Q8W1X2', 'Q9ZL00', 'B7UGB9', 'Q63SC2', 'Q88C26', 'P9WG77', 'Q1R8V2', 'Q2L1P5', 'O25515', 'Q87FP6', 'P66914', 'B7NPV5', 'Q39I40', 'Q87TZ6', 'B3H2H2', 'Q3JQA6', 'Q66A50', 'Q6NG19', 'Q1J237', 'P66915', 'Q9USL6', 'Q8FFB5', 'Q1CIM6', 'O48881', 'Q3K4B8', 'Q08224', 'Q32FD7', 'B7LCG3', 'Q4K3F6', 'Q02DJ3', 'B7V753', 'P44690', 'P40192', 'Q8Z6Q3', 'Q83K78', 'P76422', 'Q51892', 'A8A2R4', 'Q8Z4W1', 'Q6G7L7', 'C3K4G7', 'Q65UE8', 'Q06490', 'Q5PNC8', 'Q5PIK8', 'Q1C792', 'O94265', 'O83153', 'Q1AYE5', 'Q55EK9', 'Q3Z1Z2', 'C4ZVU9', 'P39988', 'Q8ZPM8', 'Q0II59', 'Q08975', 'Q4ZL75', 'Q320Z3', 'Q62LP6', 'A5UA83', 'Q6GEY2', 'Q7MGA4', 'Q1RBF9', 'B7MY71', 'B7N609', 'Q0TF48', 'Q6LP62', 'Q3YZC3', 'Q6AFC1', 'Q7W6K7', 'Q0I3D2', 'O31620', 'O85786', 'O67772', 'P39610', 'Q7WII1', 'Q6D5V1', 'B1JFM7', 'P82197', 'B1LML3', 'B7MHS2', 'P99124', 'O94266', 'A7ZPL9', 'Q48BL6', 'B2TX08', 'Q8XBL0', 'P9WG76', 'Q8FT

Find all the proteins in the target database (uniprot_sprot.fasta) -> complete ground truth (actual positives + actual negatives)

In [ ]:
path = datapath + 'uniprot_sprot.fasta'
all_sequences = SeqIO.parse(path, 'fasta')

list_all_sequences = []
for sequence in all_sequences:
    name = sequence.id # name is in the form sp|Q6GZX4|001R_FRG3G
    list_all_sequences.append(name.split('|')[1])

num_sequences = len(set(list_all_sequences))
print(f"Total number of proteins in SwissProt: {num_sequences}")

Total number of proteins in SwissProt: 563972


#### Read the output file of the psi-blast search

In [ ]:
def read_psiblast(path):
    ''' Create a dictionary having as key the protein ID and as value the set of
    tuples (start, end) representing the position of our domain in that protein
    (we assume that the domain may be present in more than one position). '''
    
    positions = {}
    with open(path, 'r') as f:
        for line in f:
            if line:
                line = line.replace('|', ' ')
                (_, _, id, _, _, length, _, gapopen, _, _, 
                    start, end, evalue, bitscore) = line.split()
                    
                positions.setdefault(id, set()).add((start, end)) 
    return positions

In [ ]:
path = datapath + 'psiblast_search_uniref90_100hits.txt'
positions_psiblast = read_psiblast(path)
print(f'Number of distint proteins found with psi-blast: {len(positions_psiblast)}')

Number of distint proteins found with psi-blast: 661


In [ ]:
positions_psiblast

{'A0A0H2WZY4': {('106', '278')},
 'A0AFC3': {('15', '229'), ('20', '217'), ('21', '229')},
 'A0KPL4': {('189', '287'), ('83', '297')},
 'A0KTG8': {('169', '287')},
 'A0LP26': {('12', '238'), ('22', '236')},
 'A0Q299': {('26', '236'), ('8', '238')},
 'A0R980': {('13', '234'), ('27', '233')},
 'A0RU82': {('28', '263'), ('29', '263'), ('30', '249')},
 'A1A6H3': {('240', '368'), ('244', '313')},
 'A1ACX0': {('22', '243'), ('25', '241')},
 'A1ADT5': {('13', '276'), ('14', '276'), ('18', '276'), ('89', '256')},
 'A1AFX4': {('124', '290'), ('86', '297'), ('95', '308')},
 'A1AV12': {('134', '298')},
 'A1JQJ2': {('18', '242'), ('24', '240'), ('68', '223')},
 'A1JQV6': {('95', '295')},
 'A1RGD1': {('174', '287')},
 'A1S0R2': {('321', '476')},
 'A1SX95': {('12', '218'), ('12', '242')},
 'A1SZY2': {('124', '296'), ('130', '291')},
 'A1TYR4': {('172', '285')},
 'A1VBV4': {('10', '199'), ('24', '199')},
 'A2BLC0': {('248', '488')},
 'A2RKJ5': {('3', '228')},
 'A2SU11': {('25', '234'), ('4', '236'), 

#### Read the output file of the hmm search

In [ ]:
def read_hmm(path):

    ''' Create a dictionary having as key the protein ID and as value the set of
    tuples (start, end) representing the position of our domain in that protein
    (we assume that the domain may be present in more than one position). '''
  
    positions = {}
    with open(path,'r') as f:
        for line in f:
            if line[0] != "#": # and line[0] != "-":
              line=line.replace('|', ' ')
              (_, id, _, _, tlen, _, _, qlen,  
              full_seq_evalue, full_seq_score, _, _, _, dcevalue, dievalue, dscore, _, _, _ , 
              align_from, align_to, env_from, env_to, accuracy, description) = line.strip().split()[:25]
              #exist multiple scores and alignment. Above I took all of them, but in the below dictionary
              #I have put the same used in github reference
            
              positions.setdefault(id, set()).add( (align_from, align_to) )
   
    return positions


In [ ]:
path = datapath + 'hmm_search_uniref90_100hits.hmmer_domtblout'
positions_hmm = read_hmm(path)
print(f'Number of distint proteins found with hmm: {len(positions_hmm)}')

Number of distint proteins found with hmm: 301


In [ ]:
positions_hmm

{'A0A0H2WZY4': {('173', '229')},
 'A0KTG8': {('189', '289')},
 'A1A6H3': {('243', '348')},
 'A1ACX0': {('69', '239')},
 'A1ADT5': {('89', '278')},
 'A1AV12': {('199', '299')},
 'A1RGD1': {('189', '289')},
 'A1S3I3': {('125', '148'), ('189', '289')},
 'A3D837': {('189', '289')},
 'A3N2D3': {('76', '270')},
 'A3NJS6': {('197', '237'), ('88', '141')},
 'A3P5E2': {('197', '237'), ('88', '141')},
 'A4VR47': {('187', '287')},
 'A4XPU9': {('188', '287')},
 'A4Y9Z9': {('189', '289')},
 'A5EWS4': {('190', '289'), ('309', '351')},
 'A5G6F4': {('199', '299')},
 'A5GMH7': {('200', '269')},
 'A5HYZ4': {('61', '145')},
 'A5N1D3': {('75', '147')},
 'A5UA83': {('76', '273')},
 'A5W9W6': {('188', '287')},
 'A5WB73': {('79', '186')},
 'A6TL31': {('247', '329')},
 'A6VDB5': {('188', '287')},
 'A6VEZ4': {('79', '260')},
 'A6VNE5': {('75', '182')},
 'A6VPG7': {('59', '206')},
 'A6W6A1': {('28', '62'), ('87', '239')},
 'A6WJS7': {('189', '289')},
 'A7FPT1': {('61', '145')},
 'A7GAL1': {('61', '160')},
 'A7N

#### Model evaluation

In [ ]:
# !pip install Bio
# from Bio import SeqIO
import math
import json
import pandas as pd
import numpy as np

#### Useful functions to compute metrics

CONFIDENTIAL!!! :-)

In $\verb!utils.py!$ there are 5 functions:

- $\verb!computeMetrics!$
- $\verb!createPositionSet!$
- $\verb!evaluateSequencesSH2!$
- $\verb!getSeqLength!$
- $\verb!evaluatePositionsSH2!$

But for point 8) are needed only:
- $\verb!computeMetrics!$
- $\verb!evaluateSequencesSH2!$

So I use only them in the following.

In [ ]:
# import math
def metrics(true_positive, true_negative, false_positive, false_negative):
  
    accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_negative + false_positive)
    precision = true_positive / (true_positive + false_positive)
    sensitivity = true_positive / (true_positive + false_negative)
    specificity = true_negative / (true_negative + false_positive)
    weighted_accuracy = (sensitivity + specificity) / 2
    mcc_numerator = (true_positive * true_negative) - (false_positive * false_negative)
    mcc_denominator_squared = (true_positive + false_positive) * (true_positive + false_negative) * (true_negative + false_positive) * (true_negative + false_negative)
    mcc = mcc_numerator / math.sqrt(mcc_denominator_squared)
    f1_score = 2 * (precision * sensitivity) / (precision + sensitivity)
    
    return {
            "Accuracy": accuracy,
            "Weighted accuracy": weighted_accuracy,
            "Precision": precision,
            "Sensitivity": sensitivity,
            "Specificity": specificity,
            "MCC": mcc,
            "F-score": f1_score
            }


In [ ]:
def metrics_results(predictions, actual_positives, ground_truth_size):
    """
    predictions = dict of the type {'A0A0H2WZY4': {('167', '228')}, ... }
    actual_positives = set of sequences containing PF08543
    ground_truth_size = number of actual positives + actual negatives (= SwissPort size)
    """
    predicted_sequences = set(predictions.keys())
    
    true_positive = len(predicted_sequences & actual_positives)
    
    # # If there aren't true postive later there will be a division by 0
    # if true_positive == 0:
    #     print("No true positive!")
    #     return
    
    false_negative = len(actual_positives) - true_positive
    false_positive = len(predicted_sequences) - true_positive
    true_negative = ground_truth_size - true_positive - false_positive - false_negative
    
    conf_matrix = pd.DataFrame(np.array([[true_positive, false_positive], [false_negative, true_negative]]), 
                               index=['Positive pred', 'Negative pred'], 
                               columns=['Actual positive', 'Actual negative'])

    return conf_matrix, metrics(true_positive, true_negative, false_positive, false_negative)  



#### Evaluate psi-blast

In [ ]:
conf_matrix_psiblast, metric_results_psiblast = metrics_results(positions_psiblast, set_protein_containing_domain, num_sequences )
print('CLASSIFICATION METRICS')
for k in metric_results_psiblast:
    print(f'{k}: {metric_results_psiblast[k]:.3}')
    
print('\nCONFUSION MATRIX')    
conf_matrix_psiblast

CLASSIFICATION METRICS
Accuracy: 0.999
Weighted accuracy: 1.0
Precision: 0.227
Sensitivity: 1.0
Specificity: 0.999
MCC: 0.476
F-score: 0.37

CONFUSION MATRIX


,Actual positive,Actual negative
Positive pred,150,511
Negative pred,0,563311


#### Evaluate hmm

In [ ]:
conf_matrix_hmm, metric_results_hmm = metrics_results(positions_hmm, set_protein_containing_domain, num_sequences )
print('CLASSIFICATION METRICS')
for k in metric_results_hmm:
    print(f'{k}: {metric_results_hmm[k]:.3}')
    
print('\nCONFUSION MATRIX')    
conf_matrix_hmm

CLASSIFICATION METRICS
Accuracy: 1.0
Weighted accuracy: 0.997
Precision: 0.495
Sensitivity: 0.993
Specificity: 1.0
MCC: 0.701
F-score: 0.661

CONFUSION MATRIX


,Actual positive,Actual negative
Positive pred,149,152
Negative pred,1,563670


### Comment on the results

## Ex 9

#### Read the ground truth file for the positions

In [ ]:
path = datapath + 'protein-matching-PF08543.json'

with open(path, "r") as file:
    position_file = json.load(file)
    
ground_truth_positions = {}
for elem in position_file:
    id = elem['metadata']['accession']
    length = elem['metadata']['length']

    ground_truth_positions.setdefault(id, dict()).update([('length', length), ('positions', set())])

    for entry in elem['entries']:
        if entry['accession']=='PF08543':
            for location in entry['entry_protein_locations']:

                position_tuple=(str(location['fragments'][0]['start']),
                                str(location['fragments'][0]['end']))

                ground_truth_positions[id]['positions'].add(position_tuple)

In [ ]:
print(len(ground_truth_positions))

150


In [ ]:
for item in ground_truth_positions.items():
  print(item)

('A1ADT5', {'length': 283, 'positions': {('88', '265')}})
('A3N2D3', {'length': 286, 'positions': {('64', '256')}})
('A5UA83', {'length': 288, 'positions': {('71', '264')}})
('A5WB73', {'length': 290, 'positions': {('74', '257')}})
('A6VEZ4', {'length': 288, 'positions': {('77', '259')}})
('A6VNE5', {'length': 286, 'positions': {('62', '259')}})
('A7N5Q6', {'length': 289, 'positions': {('75', '260')}})
('A7ZPL9', {'length': 283, 'positions': {('88', '265')}})
('A8A2R4', {'length': 283, 'positions': {('88', '265')}})
('B0KR83', {'length': 290, 'positions': {('76', '257')}})
('B0UUD2', {'length': 286, 'positions': {('64', '260')}})
('B1IX53', {'length': 283, 'positions': {('88', '265')}})
('B1JFM7', {'length': 290, 'positions': {('76', '257')}})
('B1LML3', {'length': 283, 'positions': {('88', '265')}})
('B1XA89', {'length': 283, 'positions': {('88', '265')}})
('B2JCI0', {'length': 287, 'positions': {('74', '261')}})
('B2TX08', {'length': 283, 'positions': {('88', '265')}})
('B3H2H2', {'l

#### Useful functions to compute metrics

CONFIDENTIAL!!! :-)

In $\verb!utils.py!$ there are 5 functions:

- $\verb!computeMetrics!$
- $\verb!createPositionSet!$
- $\verb!evaluateSequencesSH2!$
- $\verb!getSeqLength!$
- $\verb!evaluatePositionsSH2!$

But for point 9) are needed only:
- $\verb!computeMetrics!$
- $\verb!createPositionSet!$
- $\verb!getSeqLength!$
- $\verb!evaluatePositionsSH2!$

So I use only them in the following.

In [ ]:
def createPositionSet(position_set):
    """
    position_set = {('88', '265'), ...}
    return a set {88, 89, ..., 265, ...}
    """
    positions = set()
    for pos in position_set: # pos is a tuple of two strings
        p = [i for i in range( int(pos[0]), int(pos[1])+1 )]
        positions = positions.union(p)
    return positions

This is the original function.

I'm not able to run it because it calls a command line in Linux. 

If you can, please try to see if the below function provides the same result as my version.

I get the following error...

`Error: [blastdbcmd] DB contains no accession info.`

In [ ]:
############### ORIGINAL FUNCTION
'''
def getSeqLength(entry):
    """
    Get sequence length from reference dataset
    """   
    cmd = 'blastdbcmd -entry "{}" -db ../../data/SwissProt_humans_reference_all.fasta'.format(entry)
    results = subprocess.run(
            cmd, shell=True, universal_newlines=True, 
            stdout=subprocess.PIPE
            ).stdout
    results = results.split('\n')
    results.pop(0)
    return len("".join(results))
'''

'\ndef getSeqLength(entry):\n    """\n    Get sequence length from reference dataset\n    """   \n    cmd = \'blastdbcmd -entry "{}" -db ../../data/SwissProt_humans_reference_all.fasta\'.format(entry)\n    results = subprocess.run(\n            cmd, shell=True, universal_newlines=True, \n            stdout=subprocess.PIPE\n            ).stdout\n    results = results.split(\'\n\')\n    results.pop(0)\n    return len("".join(results))\n'

In [ ]:
path = datapath + 'uniprot_sprot.fasta'
all_sequences = SeqIO.parse(path, 'fasta')

In [ ]:
##################### MY VERSION
def getSeqLength(entry, all_sequences):

  """
  Get sequence length from reference dataset
  """ 
#   path = datapath + 'uniprot_sprot.fasta'
#   all_sequences = SeqIO.parse(path, 'fasta')

  list_all_sequences = []
  for sequence in all_sequences:
      name = sequence.id # name is in the form sp|Q6GZX4|001R_FRG3G
      name = name.split('|')[1]
      if name==entry:
        #print(sequence.seq)
        #print(len(sequence.seq))
        length=len(sequence.seq)
  return length


In [ ]:
#-------------------------DEBUG
entry='Q6GZX4'
getSeqLength(entry, all_sequences)

256

In [ ]:
def evaluatePositions(predicted_positions, ground_truth_positions):
    """
    predicted_positions: {'A0A0H2WZY4': {('162', '278')}, ...}
    ground_truth_positions: {'A1ADT5': {'length': 283, 'positions': {('88', '265')}}, ...}
    
    """
    global_tp, global_fp, global_fn, global_tn = [], [], [], []
    
    # scores per sequence
    log_scores = []

    for id in predicted_positions:
        if id in ground_truth_positions:
            sequence_length = ground_truth_positions[id]['length']
            ground_truth_positions_seq = createPositionSet(ground_truth_positions[id]['positions']) 
            ####################################################################################
            # Here there was an error: there was reference_positions = createPositionSet(reference_positions[seqid]['positions']), 
            # which modifies reference_positions! So we had correct results only for the first protein!
            ####################################################################################
            predicted_positions_seq = createPositionSet(predicted_positions[id])    

            overlapping_positions  = ground_truth_positions_seq & predicted_positions_seq

            true_positive = len(overlapping_positions)
            false_positive = len(predicted_positions_seq - overlapping_positions)
            false_negative = len(ground_truth_positions_seq -  overlapping_positions)
            true_negative  = sequence_length - true_positive - false_positive - false_negative

            global_tp.append(true_positive)
            global_fp.append(false_positive)
            global_fn.append(false_negative)
            global_tn.append(true_negative)
            
            log_scores.append(metrics(true_positive, true_negative, false_positive, false_negative))


        ##############
        # HA SENSO? ci mette una vita per calcolare la lunghezza delle sequence con getSeqLength 
        # e non so se sia utile/sensato comprendere le sequenze senza dominio in questo esercizio
        ##############
        # else:
        #     # false positive sequence
        #     predicted_positions_seq = createPositionSet(predicted_positions[id])

        #     true_positive = 0
        #     false_positive = len(predicted_positions_seq)
        #     false_negative = 0
        #     true_negative = 0
            
        #     lenSeq = getSeqLength(id)
            
        #     log_scores.append(
        #         {
        #             'Precision': 0,
        #             'Sensitivity': 0,
        #             'Specificity': lenSeq / (lenSeq + false_positive),
        #             'F1-score': 0,
        #             'MCC': 0
        #         }
        #     )

        #     list_tp.append(true_positive)
        #     list_fp.append(false_positive)
        #     list_fn.append(false_negative)
        #     list_tn.append(true_negative)
                   
    metrics_global = metrics(true_positive=sum(global_tp), true_negative=sum(global_tn), 
                             false_positive=sum(global_fp), false_negative=sum(global_fn))
    
    conf_matrix = pd.DataFrame(np.array([[sum(global_tp), sum(global_fp)], [sum(global_fn), sum(global_tn)]]), 
                               index=['Positive pred', 'Negative pred'], 
                               columns=['Actual positive', 'Actual negative'])
    
    return metrics_global, log_scores, conf_matrix

#### Evaluate psi-blast

In [ ]:
metrics_global_psiblast, log_score_psiblast, conf_matrix_psiblast_pos = evaluatePositions(positions_psiblast, ground_truth_positions)

In [ ]:
print('CLASSIFICATION METRICS')
for k in metrics_global_psiblast:
    print(f'{k}: {metrics_global_psiblast[k]:.3}')
print('\nCONFUSION MATRIX')    
conf_matrix_psiblast_pos

CLASSIFICATION METRICS
Accuracy: 0.79
Weighted accuracy: 0.71
Precision: 0.756
Sensitivity: 0.994
Specificity: 0.426
MCC: 0.553
F-score: 0.859

CONFUSION MATRIX


,Actual positive,Actual negative
Positive pred,29410,9486
Negative pred,189,7026


In [ ]:
log_score_psiblast

[{'Accuracy': 0.9673469387755103,
  'F-score': 0.9679999999999999,
  'MCC': 0.9354734388854886,
  'Precision': 0.9490196078431372,
  'Sensitivity': 0.9877551020408163,
  'Specificity': 0.9469387755102041,
  'Weighted accuracy': 0.9673469387755103},
 {'Accuracy': 0.9616858237547893,
  'F-score': 0.9606299212598425,
  'MCC': 0.9261480246189371,
  'Precision': 0.9242424242424242,
  'Sensitivity': 1.0,
  'Specificity': 0.9280575539568345,
  'Weighted accuracy': 0.9640287769784173},
 {'Accuracy': 0.9810554803788903,
  'F-score': 0.972,
  'MCC': 0.9577564505277352,
  'Precision': 0.9642857142857143,
  'Sensitivity': 0.9798387096774194,
  'Specificity': 0.9816700610997964,
  'Weighted accuracy': 0.9807543853886078},
 {'Accuracy': 0.9398496240601504,
  'F-score': 0.9682539682539683,
  'MCC': 0.41887185275721034,
  'Precision': 0.953125,
  'Sensitivity': 0.9838709677419355,
  'Specificity': 0.3333333333333333,
  'Weighted accuracy': 0.6586021505376344},
 {'Accuracy': 0.9548872180451128,
  'F-sc

#### Evaluate hmm

In [ ]:
metrics_global_hmm, log_score_hmm, conf_matrix_hmm_pos = evaluatePositions(positions_hmm, ground_truth_positions)

In [ ]:
print('CLASSIFICATION METRICS')
for k in metrics_global_hmm:
    print(f'{k}: {metrics_global_hmm[k]:.3}')
print('\nCONFUSION MATRIX')    
conf_matrix_hmm_pos

CLASSIFICATION METRICS
Accuracy: 0.902
Weighted accuracy: 0.911
Precision: 0.965
Sensitivity: 0.88
Specificity: 0.942
MCC: 0.8
F-score: 0.92

CONFUSION MATRIX


,Actual positive,Actual negative
Positive pred,25998,955
Negative pred,3555,15513


In [ ]:
log_score_hmm

[{'Accuracy': 0.9591836734693877,
  'F-score': 0.9591836734693877,
  'MCC': 0.9183673469387755,
  'Precision': 0.9591836734693877,
  'Sensitivity': 0.9591836734693877,
  'Specificity': 0.9591836734693877,
  'Weighted accuracy': 0.9591836734693877},
 {'Accuracy': 0.9726277372262774,
  'F-score': 0.9695740365111561,
  'MCC': 0.9447578734764585,
  'Precision': 0.9637096774193549,
  'Sensitivity': 0.9755102040816327,
  'Specificity': 0.9702970297029703,
  'Weighted accuracy': 0.9729036168923015},
 {'Accuracy': 0.9285714285714286,
  'F-score': 0.9617706237424547,
  'MCC': 0.420987849267374,
  'Precision': 0.956,
  'Sensitivity': 0.9676113360323887,
  'Specificity': 0.42105263157894735,
  'Weighted accuracy': 0.6943319838056681},
 {'Accuracy': 0.9256505576208178,
  'F-score': 0.9595141700404859,
  'MCC': 0.5078242011050007,
  'Precision': 0.9673469387755103,
  'Sensitivity': 0.9518072289156626,
  'Specificity': 0.6,
  'Weighted accuracy': 0.7759036144578313},
 {'Accuracy': 0.9712643678160919